# GluonTS: Hyperparameter Tuning

This notebook demonstrates how to tune hyperparameters in [GluonTS](https://ts.gluon.ai/stable/index.html) using [Optuna](https://optuna.org/). See [here](https://ts.gluon.ai/stable/tutorials/advanced_topics/hp_tuning_with_optuna.html) for the original tutorial.


## Data Loading

Load the M4 Hourly dataset.

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json

from gluonts.dataset.repository import get_dataset
from gluonts.dataset.util import to_pandas

In [8]:
from gluonts.dataset.repository import get_dataset

dataset = get_dataset("m4_hourly")

Extract the training split, test split, and metadata.

In [9]:
training_dataset = dataset.train
test_dataset = dataset.test
metadata = dataset.metadata

print(f"Number of training examples: {len(training_dataset)}")
print(f"Number of test examples: {len(test_dataset)}")
print(f"Recommended prediction horizon: {dataset.metadata.prediction_length}")
print(f"Frequency of the time series: {dataset.metadata.freq}")

Number of training examples: 414
Number of test examples: 414
Recommended prediction horizon: 48
Frequency of the time series: H


## Hyperparameter Tuning

In this example, we'll optimize the following hyperparameters for a [DeepAR estimator](https://ts.gluon.ai/stable/api/gluonts/gluonts.torch.html?highlight=deeparestimator#gluonts.torch.DeepAREstimator).
- `num_layers`
- `hidden_size` 

1. Define a function to convert a GluonTS `DataEntry` (a dict representing a time series) into a pandas [`DataFrame`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html)

In [ ]:
import pandas as pd
from gluonts.dataset.common import DataEntry


def dataentry_to_dataframe(entry: DataEntry) -> pd.DataFrame:
    """
    Converts a GluonTS DataEntry to a pandas DataFrame.

    Args:
        entry (DataEntry): A dictionary representing a time series. Must
            contain a `start` field and a `target` field.

    Returns:
        pd.DataFrame: The input time series represented as a pandas DataFrame.
    """
    return pd.DataFrame(
        entry["target"],
        columns=[entry.get("item_id")],
        index=pd.period_range(
            start=entry["start"],
            periods=len(entry["target"]),
            freq=entry["start"].freq,
        ),
    )

2. Define a class that'll be used during the tuning process. This class can be configured with the dataset, prediction length, frequency, and the metric to be used for evaluting the model.

Here's what we'll do in each method:
 - `__init__`:
   - Initialize the objective to optimize and split the dataset into two parts using GluonTS's [`split`](https://ts.gluon.ai/stable/api/gluonts/gluonts.dataset.split.html?highlight=split#module-gluonts.dataset.split) method:
     - `validation_input`: the input part used in validation
     - `validation_label`: the label part used in validation
 - `get_params`:
   - We'll define what hyperparameters will be tuned within given range
 - `__call__`:
   - We'll define how the `DeepAREstimator` is used in training and validation.

In [11]:
from gluonts.dataset.split import split
from gluonts.evaluation import Evaluator
from gluonts.torch.model.deepar import DeepAREstimator
from typing import Iterable


class DeepARTuningObjective:
    def __init__(
        self,
        dataset: Iterable[DataEntry],
        prediction_length: int,
        freq: str,
        metric_type: str = "mean_wQuantileLoss",
    ):
        self.dataset = dataset
        self.prediction_length = prediction_length
        self.freq = freq
        self.metric_type = metric_type
        self.train, test_template = split(dataset, offset=-self.prediction_length)

        # ? For your case, can you just pass the val set directly?
        validation = test_template.generate_instances(
            prediction_length=prediction_length
        )
        self.validation_input = [entry[0] for entry in validation]
        self.validation_label = [
            dataentry_to_dataframe(entry[1]) for entry in validation
        ]

    def get_params(self, trial) -> dict:
        """
        Get the parameters for the DeepAR model based on the trial.
        """
        return {
            "num_layers": trial.suggest_int("num_layers", 1, 5),
            "hidden_size": trial.suggest_int("hidden_size", 10, 50),
        }

    def __call__(self, trial):
        params = self.get_params(trial)

        # Initialize new DeepAR estimator with the suggested parameters
        estimator = DeepAREstimator(
            num_layers=params["num_layers"],
            hidden_size=params["hidden_size"],
            prediction_length=self.prediction_length,
            freq=self.freq,
            trainer_kwargs={
                "enable_progress_bar": False,
                "enable_model_summary": False,
                "max_epochs": 10,
            },
        )

        # Train estimator
        predictor = estimator.train(self.train, cache_data=True)

        # Generate forecasts
        forecasts = list(predictor.predict(self.validation_input))

        # Create new evaluator
        evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])

        # Aggregate metrics across all time series
        agg_metrics, item_metrics = evaluator(
            self.validation_label,
            forecasts,
            num_series=len(self.dataset),
        )

        return agg_metrics[self.metric_type]

3. Perform hyperparameter tuning using the Optuna tuning process.

In [12]:
import optuna
import time
import torch

start_time = time.time()

study = optuna.create_study(direction="minimize")
study.optimize(
    DeepARTuningObjective(
        training_dataset,
        prediction_length=metadata.prediction_length,
        freq=metadata.freq,
    ),
    n_trials=10,
)
end_time = time.time()

print(f"Time taken for hyperparameter tuning: {(end_time - start_time):.2f} seconds")

print(f"Best metric: {study.best_value}")

best_trial = study.best_trial
print("Best hyperparameters found:")
for key, value in best_trial.params.items():
    print(f"{key}: {value}")

[I 2025-06-30 02:10:36,801] A new study created in memory with name: no-name-785e4dfd-4e03-4c66-b293-24cc0a5583f6
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
c:\Users\Mike\anaconda3\envs\gift\Lib\site-packages\lightning\pytorch\trainer\configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
INFO: Epoch 0, global step 50: 'train_loss' reached 5.85335 (best 5.85335), saving model to 'c:\\Users\\Mike\\vscode\\gift-eval\\notebooks\\lightning_logs\\version_5\\checkpoints\\epoch=0-step=50.ckpt' as top 1
INFO:lightning.pytorch.utilities.rank_zero:Epoch 0, global step 50: 'train_loss' reached 5.85335 (best 5.85335), sav

Time taken for hyperparameter tuning: 351.02 seconds
Best metric: 0.039687599500918117
Best hyperparameters found:
num_layers: 4
hidden_size: 24


4. Now that we've finished hyperparameter tuning, we can use the best hyperparameters to re-train the model on the whole training set.

In [ ]:
# Create final model with the best hyperparameters
estimator = DeepAREstimator(
    num_layers=best_trial.params["num_layers"],
    hidden_size=best_trial.params["hidden_size"],
    prediction_length=dataset.metadata.prediction_length,
    context_length=100,
    freq=dataset.metadata.freq,
    trainer_kwargs={
        "enable_progress_bar": False,
        "enable_model_summary": False,
        "max_epochs": 10,
    },
)

# Train the final model with the best hyperparameters
predictor = estimator.train(dataset.train, cache_data=True)